### This notebook is used to calculate the estimated distance and time from property to the firestation
Env: conda create -n driving_distance python=3.12.11\
conda init bash \
source ~/.bashrc \
conda activate driving_distance\
<!---for read_data.py--->

conda install -c conda-forge fsspec s3fs=2023.6.0 pyarrow tqdm scipy aiohttp \
<!---for ask_online.py--->

conda install -c conda-forge pandas numpy boto3  requests geopy ipython \




The files directories are as follows:\
Firestations: s3://pr-home-datascience/DSwarehouse/Datasources/FireStationLocation/FireStation/FireStations_0.csv \
Properties: s3://pr-home-datascience/Users/test/IntermediateDrive/Quantarium/AD/good_address/rundt=202504/ \


In [1]:
# import the libiaries
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import os
import read_data
import functions

cwd = os.getcwd()
print(cwd)

/home/sagemaker-user/DRIVING_DISTANCE


In [2]:
# The Paths and directories
states = ['MA']
#  CT, MA, NH, NJ, NY, PA


FS_path = "s3://pr-home-datascience/DSwarehouse/Datasources/FireStationLocation/FireStation/FireStations_0.csv"

for state in states:
    property_path = "s3://pr-home-datascience/Users/test/IntermediateDrive/Quantarium/AD/good_address/rundt=202504/" + 'state='+states[0] + '/'
    df_FS = read_data.ReadData(FS_path, state = state, directory=False)

    df_property = read_data.ReadData(property_path, state=state, directory=True)

    print(f'There are {len(df_FS.data)} firestations in MA. ')
    print(f'There are {len(df_property.data)} properties in MA. ')


In [ ]:
print(len(df_FS.data['ID'].unique()))    
print(df_FS.data.keys())
print(df_property.data.keys())
print(df_property.data.head())

815
Index(['OBJECTID', 'ID', 'NAME', 'TELEPHONE', 'ADDRESS', 'ADDRESS2', 'CITY',
       'STATE', 'ZIP', 'ZIPP4', 'COUNTY', 'FIPS', 'DIRECTIONS', 'EMERGTITLE',
       'EMERGTEL', 'EMERGEXT', 'CONTDATE', 'CONTHOW', 'GEODATE', 'GEOHOW',
       'HSIPTHEMES', 'NAICSCODE', 'NAICSDESCR', 'GEOLINKID', 'X', 'Y',
       'ST_VENDOR', 'ST_VERSION', 'GEOPREC', 'PHONELOC', 'QC_QA', 'STATE_ID',
       'FDID', 'FRST_MBRS', 'EMS_MBRS', 'TOTALPERS', 'NUMTRKS', 'NUMABUL',
       'TOTAL_VEHI', 'NBR_STA', 'OWNER', 'LEVEL_', 'TYPE', 'SPECIALTY',
       'EMSLICENSE', 'EMERPHONE', 'EMS', 'PERM_ID', 'GNIS_ID', 'x2', 'y2'],
      dtype='object')
Index(['QPID', 'State', 'PA_Latitude', 'PA_Longitude', 'Match_Code',
       'Location_Code', 'add_check', 'geo_check', 'source', 'rundt', 'state'],
      dtype='object')
       QPID State  PA_Latitude  PA_Longitude  Match_Code Location_Code  \
0  40136573    MA      42.4751      -70.9358           1           AS0   
1  42438550    MA      42.1538      -71.6540          

In [ ]:
# Use the functions in the functions.py to calculate the distance and time from the properties to the nearest firestation
# Please notice, we calcualte the routes to 5 nearest firestation by default and record the best route.


# For MA, there are total 24 chucks with size 100,000, 
# This will be cut in 0-5, 6-11, 12-17, 18-24
state = 'MA'
output_path = "./results/" + state +'/'


total = len(df_property.data)
chunksize = 100000
#chunksize = 2000
process_batch_size = 150
num_chunks = (total + chunksize - 1) // chunksize

for i in range(2, 5):
    print(f'Now is running the batch {i}.\n')
    start = i * chunksize
    end = min((i + 1) * chunksize, total)
    df_chunk = df_property.data.iloc[start:end]

    # process the chunk of properties
    results_df = functions.process_all_batches(df_chunk, df_FS.data, batchsize=process_batch_size)
    output_file = output_path +  f"FS_batch_{i}_n{str(chunksize)}.csv"

    if len(results.df) == chunksize:
        results_df.to_csv(output_file, index = False)
        print(f'Finishing the writing file batch {i}.\n')
    else:
        print(f'Missing data in the batch {i}.\n')




Now is running the batch 2.



  2%|▏         | 13/667 [09:23<7:52:03, 43.31s/it]


KeyboardInterrupt: 